In [2]:
from cryptography import x509
from cryptography.x509.oid import NameOID
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from datetime import datetime, timedelta

In [3]:
# 1. Generate RSA key pair
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048
)
public_key = private_key.public_key()

In [5]:
# 2. Build a self-signed X.509 certificate
subject = issuer = x509.Name([
    x509.NameAttribute(NameOID.COUNTRY_NAME, u"IN"),
    x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, u"Tamil Nadu"),
    x509.NameAttribute(NameOID.LOCALITY_NAME, u"Chennai"),
    x509.NameAttribute(NameOID.ORGANIZATION_NAME, u"IIT Madras"),
    x509.NameAttribute(NameOID.COMMON_NAME, u"ee21b128@smail.iitm.ac.in"),
])

certificate = (
    x509.CertificateBuilder()
    .subject_name(subject)
    .issuer_name(issuer)
    .public_key(public_key)
    .serial_number(x509.random_serial_number())
    .not_valid_before(datetime.utcnow())
    .not_valid_after(datetime.utcnow() + timedelta(days=365))
    .add_extension(
        x509.BasicConstraints(ca=True, path_length=None), critical=True
    )
    .sign(private_key, hashes.SHA256())
)

In [7]:
# Serialize private key and certificate to PEM
private_pem = private_key.private_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PrivateFormat.TraditionalOpenSSL,
    encryption_algorithm=serialization.NoEncryption()
)
cert_pem = certificate.public_bytes(serialization.Encoding.PEM)

In [13]:
# 3. Sign a message
message = b"Hello, secure world!"
signature = private_key.sign(
    message,
    padding.PKCS1v15(),
    hashes.SHA256()
)
print("Signature:", signature.hex())

Signature: 62167f0ba95fb90c95dd13b3cd7e422c0801239ff24a36eadc84e8db45724eb987125807ae7051ce62a5421293508752ebc486e3d132d353eb1ab2c737e015479bcc04a0a1b1cb9bf13f1108832f13c1928e70a29d94d23164f455b457f5403fb3c90e3cb3e945c379cc3c827ec3dbb01ef9d25f1f63305394bae522aef8205382d3b6b405231bd8ecff35ddac26be6249b7704f62f09c3addc81d27ccd0170446716cc78c0c745bfb322c539b589952b465b23038b893e27ea691a0fd9263172389254c5ec65c153d26a6213f78286a02f1a944e1d4b45a4330a96d901616d978e9a733db53f0027c7a45f7101da8f704ab844eb1c4d560e0f4f1208a815de3


In [14]:
# 4. Verify signature using the public key from the certificate
public_key_from_cert = certificate.public_key()
try:
    public_key_from_cert.verify(
        signature,
        message,
        padding.PKCS1v15(),
        hashes.SHA256()
    )
    print("Verification successful: signature is valid.")
except Exception as e:
    print("Verification failed:", e)

Verification successful: signature is valid.
